# Develop run_NASEM

In [4]:
# Refactoring NASEM model
import nasem_dairy as nd
import pandas as pd
import math
from pprint import pprint


def run_NASEM():
    ########################################
    # Step 1: Read User Input
    ########################################
    # animal_input is a dictionary with all animal specific parameters
    # diet_info is a dataframe with the user entered feed ingredients and %DM intakes
    user_diet, animal_input, equation_selection = nd.read_csv_input(
        './input.csv')

    # list_of_feeds is used to query the database and retrieve the ingredient composition, stored in feed_data
    list_of_feeds = user_diet['Feedstuff'].tolist()
    feed_data = nd.fl_get_rows(
        list_of_feeds, '../../src/nasem_dairy/data/diet_database.db')
    feed_data.reset_index(inplace=True)
    feed_data = feed_data.rename(columns={"Fd_Name": "Feedstuff"})

    diet_info_initial = pd.DataFrame({'Feedstuff': user_diet['Feedstuff']})
    diet_info_initial = diet_info_initial.merge(
        feed_data, how='left', on='Feedstuff')

    # Add Fd_DMInp to the diet_info dataframe
    Fd_DMInp = user_diet.set_index(
        'Feedstuff')['kg_user'] / user_diet['kg_user'].sum()
    diet_info_initial.insert(1, 'Fd_DMInp', Fd_DMInp.reindex(
        diet_info_initial['Feedstuff']).values)
    diet_info_initial['Fd_DMIn'] = diet_info_initial['Fd_DMInp'] * \
        animal_input['DMI']
    # Should this be done after DMI equations?

    # Add Fd_DNDF48 column, need to add to the database
    diet_info_initial['Fd_DNDF48'] = 0

    # Calculate additional physiology values
    animal_input['An_PrePartDay'] = animal_input['An_GestDay'] - \
        animal_input['An_GestLength']
    animal_input['An_PrePartWk'] = animal_input['An_PrePartDay'] / 7

    del (list_of_feeds, Fd_DMInp)

    # Create infusion_input
    if equation_selection['use_infusions'] == 0:
        # If no infusions then all values are set to 0
        no_infusion_input = {'Inf_Acet_g': 0,
                             'Inf_ADF_g': 0,
                             'Inf_Arg_g': 0,
                             'Inf_Ash_g': 0,
                             'Inf_Butr_g': 0,
                             'Inf_CP_g': 0,
                             'Inf_CPARum_CP': 0,
                             'Inf_CPBRum_CP': 0,
                             'Inf_CPCRum_CP': 0,
                             'Inf_dcFA': 0,
                             'Inf_dcRUP': 0,
                             'Inf_DM_g': 0,
                             'Inf_EE_g': 0,
                             'Inf_FA_g': 0,
                             'Inf_Glc_g': 0,
                             'Inf_His_g': 0,
                             'Inf_Ile_g': 0,
                             'Inf_KdCPB': 0,
                             'Inf_Leu_g': 0,
                             'Inf_Lys_g': 0,
                             'Inf_Met_g': 0,
                             'Inf_NDF_g': 0,
                             'Inf_NPNCP_g': 0,
                             'Inf_Phe_g': 0,
                             'Inf_Prop_g': 0,
                             'Inf_St_g': 0,
                             'Inf_Thr_g': 0,
                             'Inf_Trp_g': 0,
                             'Inf_ttdcSt': 0,
                             'Inf_Val_g': 0,
                             'Inf_VFA_g': 0,
                             'Inf_Location': 0
                             }
    elif equation_selection['use_infusions'] == 1:
        infusion_input = nd.read_infusion_input('./infusion_input.csv')
    else:
        raise ValueError(
            f"Invalid use_infusions: {equation_selection['use_infusions']} was entered. Must be 0 or 1")

########################################
# Step 1.5: Input Based Equations
########################################
    K_FeCPend_ClfLiq = nd.calculate_K_FeCPend_ClfLiq(animal_input['An_StatePhys'],
                                                     equation_selection['NonMilkCP_ClfLiq'])

########################################
# Step 2: DMI Equations
########################################
    # # Need to precalculate Dt_NDF for DMI predicitons, this will be based on the user entered DMI (animal_input['DMI])
    Dt_NDF = (diet_info_initial['Fd_NDF'] *
              diet_info_initial['Fd_DMInp']).sum()

    if equation_selection['DMIn_eqn'] == 0:
        # print('Using user input DMI')
        pass

    # Predict DMI for lactating cow
    elif equation_selection['DMIn_eqn'] == 8:
        # print("using DMIn_eqn: 8")
        animal_input['DMI'] = nd.calculate_Dt_DMIn_Lact1(
            animal_input['An_Parity_rl'],
            animal_input['Trg_MilkProd'],
            animal_input['An_BW'],
            animal_input['An_BCS'],
            animal_input['An_LactDay'],
            animal_input['Trg_MilkFatp'],
            animal_input['Trg_MilkTPp'],
            animal_input['Trg_MilkLacp'])

    # Predict DMI for heifers
    elif equation_selection['DMIn_eqn'] in [2, 3, 4, 5, 6, 7, 12, 13, 14, 15, 16, 17]:
        animal_input['DMI'] = nd.heifer_growth(
            equation_selection['DMIn_eqn'],
            # diet_info.loc['Diet', 'Fd_NDF'],
            Dt_NDF,
            animal_input['An_BW'],
            animal_input['An_BW_mature'],
            animal_input['An_PrePartWk'],
            nd.coeff_dict)

    elif equation_selection['DMIn_eqn'] in [10, 11]:
        animal_input['DMI'] = nd.dry_cow_equations(
            equation_selection['DMIn_eqn'],
            animal_input['An_BW'],
            animal_input['An_PrePartWk'],
            animal_input['An_GestDay'],
            animal_input['An_GestLength'],
            Dt_NDF,
            nd.coeff_dict)

    else:
        # It needs to catch all possible solutions, otherwise it's possible that it stays unchanged without warning
        print("DMIn_eqn uncaught - DMI not changed. equation_selection[DMIn_eqn]: " + str(
            equation_selection['DMIn_eqn']))

    # Calculated again as part of diet_data, value may change depending on DMIn_eqn selections
    del (Dt_NDF)

########################################
# Step 3: Feed Based Calculations
########################################
    # Calculate An_DMIn_BW with the final DMI value; required for calculating diet_data_initial
    An_DMIn_BW = nd.calculate_An_DMIn_BW(animal_input['An_BW'],
                                         animal_input['DMI'])
    Fe_rOMend = nd.calculate_Fe_rOMend(animal_input['DMI'],
                                       nd.coeff_dict)

    diet_info = nd.calculate_diet_info(animal_input['DMI'],
                                       animal_input['An_StatePhys'],
                                       equation_selection['Use_DNDF_IV'],
                                       diet_info=diet_info_initial,
                                       coeff_dict=nd.coeff_dict)
    # All equations in the f dataframe go into calculate_diet_info()
    # This includes micronutrient calculations which are no longer handled by seperate functions

    diet_data_initial = nd.calculate_diet_data_initial(diet_info,
                                                       animal_input['DMI'],
                                                       animal_input['An_BW'],
                                                       animal_input['An_StatePhys'],
                                                       An_DMIn_BW,
                                                       Fe_rOMend,
                                                       nd.coeff_dict)
    # diet_data contains everything starting with "Dt_"

########################################
# Step 4: Infusion Calculations
########################################
    if equation_selection['use_infusions'] == 0:
        infusion_data = nd.calculate_infusion_data(no_infusion_input,
                                                   animal_input['DMI'],
                                                   nd.coeff_dict)
    elif equation_selection['use_infusions'] == 1:
        infusion_data = nd.calculate_infusion_data(infusion_input,
                                                   animal_input['DMI'],
                                                   nd.coeff_dict)
    else:
        raise ValueError(
            f"Invalid use_infusions: {equation_selection['use_infusions']} was entered. Must be 0 or 1")

########################################
# Step 5: Animal Level Calculations
########################################
# Combine Diet and Infusion nutrient supplies
    An_data_initial = nd.calculate_An_data_initial(animal_input,
                                                   diet_data_initial,
                                                   infusion_data,
                                                   equation_selection['Monensin_eqn'],
                                                   nd.coeff_dict)

########################################
# Step 6: Rumen Digestion Calculations
########################################
    # Rumen Digestability Coefficients
    Rum_dcNDF = nd.calculate_Rum_dcNDF(animal_input['DMI'],
                                       diet_data_initial['Dt_NDFIn'],
                                       diet_data_initial['Dt_StIn'],
                                       diet_data_initial['Dt_CPIn'],
                                       diet_data_initial['Dt_ADFIn'],
                                       diet_data_initial['Dt_ForWet'])

    Rum_dcSt = nd.calculate_Rum_dcSt(animal_input['DMI'],
                                     diet_data_initial['Dt_ForNDF'],
                                     diet_data_initial['Dt_StIn'],
                                     diet_data_initial['Dt_ForWet'])

    # Rumen Digestable Intakes
    Rum_DigNDFIn = nd.calculate_Rum_DigNDFIn(Rum_dcNDF,
                                             diet_data_initial['Dt_NDFIn'])

    Rum_DigStIn = nd.calculate_Rum_DigStIn(Rum_dcSt,
                                           diet_data_initial['Dt_StIn'])

########################################
# Step 7: Microbial Protein Calculations
########################################
    if equation_selection['MiN_eqn'] == 1:
        RDPIn_MiNmax = nd.calculate_RDPIn_MiNmax(animal_input['DMI'],
                                                 An_data_initial['An_RDP'],
                                                 An_data_initial['An_RDPIn'])
        MiN_Vm = nd.calculate_MiN_Vm(RDPIn_MiNmax,
                                     nd.coeff_dict)
        Du_MiN_g = nd.calculate_Du_MiN_NRC2021_g(MiN_Vm,
                                                 Rum_DigNDFIn,
                                                 Rum_DigStIn,
                                                 An_data_initial['An_RDPIn_g'],
                                                 nd.coeff_dict)
    elif equation_selection['MiN_eqn'] == 2:
        Du_MiN_g = nd.calculate_Du_MiN_VTln_g(diet_data_initial['Dt_rOMIn'],
                                              diet_data_initial['Dt_ForNDFIn'],
                                              An_data_initial['An_RDPIn'],
                                              Rum_DigStIn,
                                              Rum_DigNDFIn,
                                              nd.coeff_dict)
    elif equation_selection['MiN_eqn'] == 3:
        Du_MiN_g = nd.calculate_Du_MiN_VTnln_g(An_data_initial['An_RDPIn'],
                                               Rum_DigNDFIn,
                                               Rum_DigStIn)
    else:
        raise ValueError(
            f"Invalid MiN_eqn: {equation_selection['MiN_eqn']} was entered. Must choose 1, 2 or 3.")

    Du_MiCP_g = nd.calculate_Du_MiCP_g(Du_MiN_g)
    Du_MiTP_g = nd.calculate_Du_MiTP_g(Du_MiCP_g, nd.coeff_dict)
    Du_MiCP = nd.calculate_Du_MiCP(Du_MiCP_g)
    Du_idMiCP_g = nd.calculate_Du_idMiCP_g(Du_MiCP_g,
                                           nd.coeff_dict)
    Du_idMiCP = nd.calculate_Du_idMiCP(Du_idMiCP_g)

########################################
# Step 7.1: Fe_CP Calculation /  Finish Dt_ and An_ calculations
########################################
# Required to finish Dt_ and An_ calculations
    Fe_RUP = nd.calculate_Fe_RUP(An_data_initial['An_RUPIn'],
                                 infusion_data['InfSI_TPIn'],
                                 An_data_initial['An_idRUPIn'])
    Fe_RumMiCP = nd.calculate_Fe_RumMiCP(Du_MiCP,
                                         Du_idMiCP)
    Fe_CPend_g = nd.calculate_Fe_CPend_g(animal_input['An_StatePhys'],
                                         An_data_initial['An_DMIn'],
                                         An_data_initial['An_NDF'],
                                         animal_input['DMI'],
                                         diet_data_initial['Dt_DMIn_ClfLiq'],
                                         K_FeCPend_ClfLiq)
    Fe_CPend = nd.calculate_Fe_CPend(Fe_CPend_g)
    Fe_CP = nd.calculate_Fe_CP(animal_input['An_StatePhys'],
                               diet_data_initial['Dt_CPIn_ClfLiq'],
                               diet_data_initial['Dt_dcCP_ClfDry'],
                               An_data_initial['An_CPIn'],
                               Fe_RUP,
                               Fe_RumMiCP,
                               Fe_CPend,
                               infusion_data['InfSI_NPNCPIn'],
                               nd.coeff_dict)

########################################
# Step 7.2: Complete diet_data and An_data
########################################
    diet_data = nd.calculate_diet_data_complete(diet_data_initial,
                                                animal_input['An_StatePhys'],
                                                Fe_CP,
                                                equation_selection,
                                                nd.coeff_dict)
    An_data = nd.calculate_An_data_complete(An_data_initial,
                                            diet_data,
                                            animal_input['An_StatePhys'],
                                            Fe_CP,
                                            infusion_data,
                                            equation_selection,
                                            nd.coeff_dict)

########################################
# Step 10: Metabolizable Protein Intake
########################################
    Du_idMiTP_g = nd.calculate_Du_idMiTP_g(Du_idMiCP_g,
                                           nd.coeff_dict)
    Du_idMiTP = nd.calculate_Du_idMiTP(Du_idMiTP_g)
    An_MPIn = nd.calculate_An_MPIn(diet_data['Dt_idRUPIn'],
                                   Du_idMiTP)
    An_MPIn_g = nd.calculate_An_MPIn_g(An_MPIn)

########################################
# Step 8: Amino Acid Calculations
########################################
    AA_list = ['Arg', 'His', 'Ile', 'Leu',
               'Lys', 'Met', 'Phe', 'Thr', 'Trp', 'Val']
    AA_values = pd.DataFrame(index=AA_list)
    # Dataframe for storing all individual amino acid values

    AA_values['Du_AAMic'] = nd.calculate_Du_AAMic(Du_MiTP_g,
                                                  AA_list,
                                                  nd.coeff_dict)
    AA_values['Du_IdAAMic'] = nd.calculate_Du_IdAAMic(AA_values['Du_AAMic'],
                                                      nd.coeff_dict)
    AA_values['Abs_AA_g'] = nd.calculate_Abs_AA_g(diet_data,
                                                  AA_values['Du_IdAAMic'],
                                                  AA_list)
    AA_values['mPrtmx_AA'] = nd.calculate_mPrtmx_AA(AA_list,
                                                    nd.coeff_dict)
    f_mPrt_max = nd.calculate_f_mPrt_max(animal_input['An_305RHA_MlkTP'],
                                         nd.coeff_dict)
    AA_values['mPrtmx_AA2'] = nd.calculate_mPrtmx_AA2(AA_values['mPrtmx_AA'],
                                                      f_mPrt_max)
    AA_values['AA_mPrtmx'] = nd.calculate_AA_mPrtmx(AA_list,
                                                    nd.coeff_dict)
    AA_values['mPrt_AA_01'] = nd.calculate_mPrt_AA_01(AA_values['AA_mPrtmx'],
                                                      AA_list,
                                                      nd.coeff_dict)
    AA_values['mPrt_k_AA'] = nd.calculate_mPrt_k_AA(AA_values['mPrtmx_AA2'],
                                                    AA_values['mPrt_AA_01'],
                                                    AA_values['AA_mPrtmx'])
    Abs_EAA_g = nd.calculate_Abs_EAA_g(AA_values['Abs_AA_g'])
    Abs_neAA_g = nd.calculate_Abs_neAA_g(An_MPIn_g,
                                         Abs_EAA_g)
    Abs_OthAA_g = nd.calculate_Abs_OthAA_g(Abs_neAA_g,
                                           AA_values['Abs_AA_g'])
    Abs_EAA2b_g = nd.calculate_Abs_EAA2b_g(equation_selection['mPrt_eqn'],
                                           AA_values['Abs_AA_g'])
    mPrt_k_EAA2 = nd.calculate_mPrt_k_EAA2(AA_values.loc['Met', 'mPrtmx_AA2'],
                                           AA_values.loc['Met', 'mPrt_AA_01'],
                                           AA_values.loc['Met', 'AA_mPrtmx'])


########################################
# Step 9: Uterine Bodyweight Calculations
########################################
    Uter_Wtpart = nd.calculate_Uter_Wtpart(
        animal_input['Fet_BWbrth'], nd.coeff_dict)
    Uter_Wt = nd.calculate_Uter_Wt(animal_input['An_Parity_rl'],
                                   animal_input['An_AgeDay'],
                                   animal_input['An_LactDay'],
                                   animal_input['An_GestDay'],
                                   animal_input['An_GestLength'],
                                   Uter_Wtpart,
                                   nd.coeff_dict)
    GrUter_Wtpart = nd.calculate_GrUter_Wtpart(
        animal_input['Fet_BWbrth'], nd.coeff_dict)
    GrUter_Wt = nd.calculate_GrUter_Wt(animal_input['An_GestDay'],
                                       animal_input['An_GestLength'],
                                       Uter_Wt,
                                       GrUter_Wtpart,
                                       nd.coeff_dict)
    Uter_BWgain = nd.calculate_Uter_BWgain(animal_input['An_LactDay'],
                                           animal_input['An_GestDay'],
                                           animal_input['An_GestLength'],
                                           Uter_Wt,
                                           nd.coeff_dict)
    GrUter_BWgain = nd.calculate_GrUter_BWgain(animal_input['An_LactDay'],
                                               animal_input['An_GestDay'],
                                               animal_input['An_GestLength'],
                                               GrUter_Wt,
                                               Uter_BWgain,
                                               nd.coeff_dict)

########################################
# Step 11: Milk Production Prediciton
########################################
    Mlk_NP_g = nd.calculate_Mlk_NP_g(animal_input['An_StatePhys'],
                                     animal_input['An_BW'],
                                     AA_values['Abs_AA_g'],
                                     AA_values['mPrt_k_AA'],
                                     Abs_neAA_g,
                                     Abs_OthAA_g,
                                     Abs_EAA2b_g,
                                     mPrt_k_EAA2,
                                     An_data['An_DigNDF'],
                                     An_data['An_DEInp'],
                                     An_data['An_DEStIn'],
                                     An_data['An_DEFAIn'],
                                     An_data['An_DErOMIn'],
                                     An_data['An_DENDFIn'],
                                     nd.coeff_dict)
    return animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data


# ### RUN MODEL ###
animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data = run_NASEM()

# Refactor calculate_An_NE

In [1]:
import numpy as np
# NOTE requires check_coeff_in_coeff_dict 

def calculate_Scrf_CP_g(An_BW):
    Scrf_CP_g = 0.20 * An_BW**0.60  # Line 1965
    return Scrf_CP_g


def calculate_Mlk_CP_g(Mlk_NP_g):
    Mlk_CP_g = Mlk_NP_g / 0.95  # Line 2213
    return Mlk_CP_g


def calculate_CPGain_FrmGain(An_BW, An_BW_mature):
    CPGain_FrmGain = 0.201 - 0.081 * An_BW / An_BW_mature   # Line 2458
    return CPGain_FrmGain


def calculate_Frm_Gain(Trg_FrmGain):
    Frm_Gain = Trg_FrmGain		#kg/d.  Add any predictions of ADG and select Trg or Pred ADG here
    return Frm_Gain


def calculate_Frm_Gain_empty(Frm_Gain, Dt_DMIn_ClfLiq, Dt_DMIn_ClfStrt, coeff_dict):
    req_coeff = ['An_GutFill_BW']
    nd.check_coeffs_in_coeff_dict(coeff_dict, req_coeff)
    Frm_Gain_empty = Frm_Gain * (1 - coeff_dict['An_GutFill_BW'])   #Line 2439, Assume the same gut fill for frame gain
    condition = (Dt_DMIn_ClfLiq > 0) and (Dt_DMIn_ClfStrt > 0)
    Frm_Gain_empty = np.where(condition, Frm_Gain * 0.91, Frm_Gain_empty)  #Line 2440, slightly different for grain & milk fed
    return Frm_Gain_empty


def calculate_NPGain_FrmGain(CPGain_FrmGain, coeff_dict):
    req_coeff = ['Body_NP_CP']
    nd.check_coeffs_in_coeff_dict(req_coeff, coeff_dict)
    NPGain_FrmGain = CPGain_FrmGain * coeff_dict['Body_NP_CP']  # Line 2460, convert to CP to TP gain / gain
    return NPGain_FrmGain


def calculate_Frm_NPgain(An_StatePhys, NPGain_FrmGain, Frm_Gain_empty, Body_Gain_empty, An_REgain_Calf):
    Frm_NPgain = NPGain_FrmGain * Frm_Gain_empty     # Line 2461
    Frm_NPgain = np.where(An_StatePhys == "Calf", (166.22 * Body_Gain_empty + 6.13 * An_REgain_Calf / Body_Gain_empty) / 1000, Frm_NPgain)
    return Frm_NPgain


def calculate_NPGain_RsrvGain(coeff_dict):
    req_coeff = ['CPGain_RsrvGain', 'Body_NP_CP']
    nd.check_coeffs_in_coeff_dict(req_coeff, coeff_dict)
    NPGain_RsrvGain = coeff_dict['CPGain_RsrvGain'] * coeff_dict['Body_NP_CP']                   # Line 2467
    return NPGain_RsrvGain


def calculate_Rsrv_Gain(Trg_RsrvGain):
    Rsrv_Gain = Trg_RsrvGain    # Line 2435
    return Rsrv_Gain


def calculate_Rsrv_Gain_empty(Rsrv_Gain):
    Rsrv_Gain_empty = Rsrv_Gain    # Line 2441, Assume no gut fill associated with reserves gain
    return Rsrv_Gain_empty


def calculate_Rsrv_NPgain(NPGain_RsrvGain, Rsrv_Gain_empty):
    Rsrv_NPgain = NPGain_RsrvGain * Rsrv_Gain_empty                    # Line 2468
    return Rsrv_NPgain


def calculate_Body_NPgain(Frm_NPgain, Rsrv_NPgain):
    Body_NPgain = Frm_NPgain + Rsrv_NPgain      # Line 2473
    return Body_NPgain


def calculate_Body_CPgain(Body_NPgain, coeff_dict):
    req_coeff = ['Body_NP_CP']
    nd.check_coeffs_in_coeff_dict(req_coeff, coeff_dict)
    Body_CPgain = Body_NPgain / coeff_dict['Body_NP_CP']     # Line 2475
    return Body_CPgain


def calculate_Body_CPgain_g(Body_CPgain):
    Body_CPgain_g = Body_CPgain * 1000     # Line 2477
    return Body_CPgain_g


def calculate_Gest_NCPgain_g(GrUter_BWgain, coeff_dict):
    req_coeff = ['CP_GrUtWt']
    nd.check_coeffs_in_coeff_dict(req_coeff, coeff_dict)
    Gest_NCPgain_g = GrUter_BWgain * coeff_dict['CP_GrUtWt'] * 1000
    return Gest_NCPgain_g


def calculate_Gest_NPgain_g(Gest_NCPgain_g, coeff_dict):
    req_coeff = ['Body_NP_CP']
    nd.check_coeffs_in_coeff_dict(req_coeff, coeff_dict)
    Gest_NPgain_g = Gest_NCPgain_g * coeff_dict['Body_NP_CP']
    return Gest_NPgain_g


def calculate_Gest_NPuse_g(Gest_NPgain_g, coeff_dict):
    req_coeff = ['Gest_NPother_g']
    nd.check_coeffs_in_coeff_dict(req_coeff, coeff_dict)
    Gest_NPuse_g = Gest_NPgain_g + coeff_dict['Gest_NPother_g']   # Line 2366
    return Gest_NPuse_g


def calculate_Gest_CPuse_g(Gest_NPuse_g, coeff_dict):
    req_coeff = ['Body_NP_CP']
    nd.check_coeffs_in_coeff_dict(req_coeff, coeff_dict)
    Gest_CPuse_g = Gest_NPuse_g / coeff_dict['Body_NP_CP']   # Line 2367
    return Gest_CPuse_g


def calculate_Ur_Nout_g(Dt_CPIn, Fe_CP, Scrf_CP_g,  Fe_CPend_g, Mlk_CP_g, Body_CPgain_g, Gest_CPuse_g):
    Ur_Nout_g = (Dt_CPIn * 1000 - Fe_CP * 1000 - Scrf_CP_g - Fe_CPend_g - Mlk_CP_g - Body_CPgain_g - Gest_CPuse_g) / 6.25     # Line 2742
    return Ur_Nout_g


def calculate_Ur_DEout(Ur_Nout_g):
    Ur_DEout = 0.0143 * Ur_Nout_g  # Line 2748
    return Ur_DEout


def calculate_An_MEIn(An_StatePhys, An_DEIn, An_GasEOut, Ur_DEout, Dt_DMIn_CflLiq, Dt_DEIn_base_ClfLiq, Dt_DEIn_base_ClfDry,K_DE_ME_ClfDry):
    An_MEIn = An_DEIn - An_GasEOut - Ur_DEout   # Line 2753
    condition = (An_StatePhys == "Calf") and (Dt_DMIn_CflLiq > 0)
    An_MEIn = np.where(condition,
                       Dt_DEIn_base_ClfLiq * 0.96 + Dt_DEIn_base_ClfDry * K_DE_ME_ClfDry, # Line 2757, no consideration of infusions for calves.
                       An_MEIn)
    return An_MEIn


def calculate_An_NEIn(An_MEIn):
    An_NEIn = An_MEIn * 0.66    # Line 2762
    return An_NEIn


def calculate_An_NE(An_NEIn, An_DMIn):
    An_NE = An_NEIn / An_DMIn   # Line 2763
    return An_NE


In [5]:
Scrf_CP_g = calculate_Scrf_CP_g(animal_input['An_BW'])

Mlk_CP_g = calculate_Mlk_CP_g(1040.4902711800644)

CPGain_FrmGain = calculate_CPGain_FrmGain(animal_input['An_BW'], 
                                          animal_input['An_BW_mature'])

Frm_Gain = calculate_Frm_Gain(animal_input['Trg_FrmGain'])

Frm_Gain_empty = calculate_Frm_Gain_empty(Frm_Gain,
                                          diet_data['Dt_DMIn_ClfLiq'],
                                          diet_data['Dt_DMIn_ClfStrt'],
                                          nd.coeff_dict)
# NOTE Start by adding Dt_DMIn_ClfStrt

KeyError: 'Dt_DMIn_ClfStrt'